# Building an End-to-End Web Application Integrated with Microsoft's Semantic Kernel and a Large Language Model

## Notebook \#1: Getting to know Microsoft Semantic Kernel & OpenAI LLMs

Welcome to the first demo notebook for our workshop on MS Semantic Kernel! SK provides a framework for developing applications that invoke LLMs. Connections to OpenAI, Azure OpenAI, and HuggingFace models are supported. We will be demonstrating SK in conjunction with OpenAI models.

**For more information, please refer to:**
- Semantic Kernel on GitHub: https://github.com/microsoft/semantic-kernel
- Semantic Kernel documentation: https://learn.microsoft.com/en-gb/semantic-kernel/ (Note that this focuses on the C# implementation)

**Requirements:**
- python >= 3.8
- semantic_kernel (pip installable!)
- An OpenAI API key (all new users have free credits to use in their first 3 months)

If you have an OpenAI platform account, you can generate a new API key here: https://platform.openai.com/account/api-keys

## Imports

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion, OpenAIChatCompletion
from semantic_kernel.utils.settings import openai_settings_from_dot_env

import asyncio

## Initializing the kernel

In [3]:
kernel = sk.Kernel()

In [4]:
# You should edit the example .env file with your own OpenAI API key before running this command
api_key, org_id = openai_settings_from_dot_env()

### Configuring OpenAI backend

Recommended model options:
- `text-davinci-003`: Most powerful InstructGPT model, \$0.02 per 1000 tokens (~750 words)
- `gpt-3.5-turbo`: Model behind ChatGPT, \$0.002 per 1000 tokens

The full list of OpenAI models and pricing is available here: https://openai.com/pricing

In [5]:
## 2 cents per 1000 tokens (prompts and responses)
#kernel.add_text_completion_service(
#    "text-davinci-003", OpenAITextCompletion("text-davinci-003", api_key, org_id)
#)

In [6]:
# 0.2 cents per 1000 tokens (prompts and responses)
kernel.add_chat_service(
    "gpt-3.5-turbo", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id)
)

## Testing the language model connection

Before getting into designing semantic skills and prompt engineering, we should confirm that we have the kernel and model connection configuration set up properly. A short question with a short response is all that is needed here. 

In [7]:
# Edit with your favourite book or series (released before 2021)
prompt = "Who is the author of the Foundation series?"

test_func = kernel.create_semantic_function(prompt, max_tokens=100, temperature=0.2)

In [8]:
answer = test_func()
print(answer)

The author of the Foundation series is Isaac Asimov.


## Building semantic functions & skills

SK encourages LLM prompt templating and reuse by bundling them into "semantic functions." Prompts are regular strings and can contain zero, one, or multiple context variables indicated by double curly braces `{{}}` and `$variable_name`. When there is only one context variable, indicate it with `{{$input}}`. 

In [9]:
# Let's start with a simple prompt -- asking the LLM to write an email about a topic

prompt = """
Write an email about {{$input}}
"""

email_writer = kernel.create_semantic_function(prompt, max_tokens=1000, temperature=0.3)

In [11]:
input_text = "a free community event in High Park"

output = email_writer(input_text)

print(output)

Subject: Join us for a Free Community Event in High Park!

Dear [Recipient],

We are excited to invite you to a free community event in High Park on [Date] from [Time] to [Time]. This event is open to everyone in the community and we hope to see you there!

The event will take place at the High Park Amphitheatre and will feature a variety of activities for all ages. There will be live music, food vendors, face painting, and much more. We will also have a raffle with some great prizes, so be sure to enter for your chance to win.

This event is a great opportunity to meet your neighbors and enjoy a fun day out in the park. We encourage you to bring your family and friends and make a day of it. There will be plenty of space to relax and enjoy the beautiful surroundings of High Park.

We hope to see you there and look forward to spending a fun-filled day with you. If you have any questions or would like more information about the event, please don't hesitate to contact us.

Thank you for y

### Aside: Prompt engineering

Unless you are very lucky, you're not likely to get the perfect response to your request from the LLM on your first try. This is where prompt engineering comes in: experimenting with changes to your prompt to improve the model's response. Even small changes to the wording or formatting of a prompt can lead to very different model responses. A prompt that works well with one model may give totally different responses when used with another model. 

This isn't unique to using SK. Anytime you use or work with LLMs, you'll likely need to do some prompt engineering.

Some approaches you can try with prompt engineering:
  - Set the tone or explain the context
  - Give specific requirements
  - Add instructions for edge cases
  - Show example inputs and responses ("few shot learning")
  
We will test these out with our basic email writing function below.

In [17]:
# Setting the tone
prompt = """
Draft an email from a local politician about {{$input}}
"""

email_writer = kernel.create_semantic_function(prompt, max_tokens=1000, temperature=0.3)
output = email_writer(input_text)

print(output)

Subject: Join us for a Free Community Event in High Park!

Dear [Recipient],

I am excited to invite you to a free community event in High Park on [Date] from [Time] to [Time]. This event is aimed at bringing together the residents of our community to celebrate our beautiful park and the people who make it special.

The event will feature a range of activities for all ages, including live music, food trucks, face painting, and much more. You will also have the opportunity to meet and interact with your neighbors and local officials, including myself.

High Park is a true gem in our community, and this event is a great way to showcase its beauty and bring people together. I encourage you to come out and enjoy the festivities with your family and friends.

Please feel free to share this invitation with your friends and family. We look forward to seeing you there!

Sincerely,

[Your Name]

[Your Title]

[Your Contact Information]


In [18]:
# Clarify instructions by giving specific requirements
prompt = """
Draft an email from a local politician about {{$input}}. 
Address the email to 'Dear neighbours'. Include links to relevant websites.
"""

email_writer = kernel.create_semantic_function(prompt, max_tokens=1000, temperature=0.3)
output = email_writer(input_text)

print(output)

Subject: Join me for a free community event in High Park!

Dear neighbours,

I am thrilled to invite you to a free community event in High Park on Saturday, August 14th from 11am-3pm. This event is a great opportunity for us to come together as a community and enjoy the beautiful outdoors while participating in fun activities and learning about local organizations.

There will be something for everyone at this event, including live music, food trucks, face painting, and a bouncy castle for the kids. You can also take part in guided nature walks, learn about local wildlife, and meet representatives from community organizations who will be on hand to answer your questions and provide information about their services.

This event is being organized by the City of Toronto in partnership with local community groups, and I am proud to support it. It's a great way for us to celebrate our community and all the wonderful things that make it special.

To learn more about the event and to RSVP, p

In [19]:
# Add instructions for edge cases
prompt = """
Draft an email from a local politician about {{$input}}. 
If the topic above is not relevant to community updates or events,
write 'Irrelevant topic' instead of an email.
"""

bad_input = "tuna casserole"

email_writer = kernel.create_semantic_function(prompt, max_tokens=1000, temperature=0.3)
output = email_writer(bad_input)

print(output)

Irrelevant topic.


In [20]:
good_input = "youth sports registration"

email_writer = kernel.create_semantic_function(prompt, max_tokens=1000, temperature=0.3)
output = email_writer(good_input)

print(output)

Subject: Register for Youth Sports Today!

Dear Community Members,

I hope this email finds you well. As we approach the end of summer, I wanted to remind you that registration for youth sports is now open. This is a great opportunity for your children to stay active, learn new skills, and make new friends.

Our community offers a variety of sports programs for children of all ages, including soccer, basketball, baseball, and more. These programs are run by dedicated volunteers who are committed to providing a safe and fun environment for our youth.

I encourage you to register your children for these programs as soon as possible. Not only will they benefit from the physical activity and social interaction, but they will also learn important life skills such as teamwork, sportsmanship, and perseverance.

If you have any questions or concerns about the registration process, please do not hesitate to reach out to me or the program coordinators. We are here to help and ensure that every c

In [22]:
# Show example inputs and model responses (few-shot learning)
# We're able to change the output style and length significantly by showing examples!

prompt = """
Write a short email from a local politician about a topic or event of local interest.

Topic: ```Recycling```

Email:
Dear neighbours,

Did you know that over 50% of the garbage collected in our city could have been recycled?
You can learn this and other surprising trash facts in this month's edition of the city
newsletter. Be sure to stick the waste sorting cheat sheet to your fridge for easy reference.
You can also visit our website to look up the correct bin for hundreds of waste items.
Together, we can cut down on landfill usage!

Sincerely,

[Deputy Mayor]

Topic: ```Canada Day Fireworks```

Email:
Dear neighbours, 

Happy Canada Day! We are holding a celebration at City Hall starting at 3 PM. Fireworks
will begin at 9:30 PM. There will be food, local vendors, musical performances, and lots
more. Everyone is welcome. Admission is free but donations to the Daily Bread Food Bank
would be appreciated.

See you there!

[Your city councillor]

Topic: ```{{$input}}```

Email:
"""

input_text = "Community event in High Park"

email_writer = kernel.create_semantic_function(prompt, max_tokens=1000, temperature=0.3)
output = email_writer(input_text)

print(output)


Dear residents,

I wanted to let you know about an exciting community event happening in High Park next weekend. On Saturday, we will be hosting a family-friendly picnic with games, music, and food. This is a great opportunity to meet your neighbours and enjoy the beautiful park together. The event will start at 12 PM and go until 4 PM. Please bring your own blankets and chairs. We hope to see you there!

Best regards,

[Your local councillor]


### Adjusting other settings

The semantic functions have several optional arguments:
- ``max_tokens`` sets the maximum response length. This is helpful for preventing overly long responses to keep costs under control. 
- ``temperature`` affects how words are selected when generating a response. A temperature of zero will always give the most likely response. Increasing the temperature allows for words other than the highest probability next words to be chosen. High temperatures can be thought of as more creative and unpredictable. 
- ``top_p`` determines the set of words that can be present in the response based on how likely they are to appear. E.g. ``top_p = 0.5`` will limit the response to the top 50\% most likely words. When set to 0 or 1 (default value), no restrictions are applied on the set of possible words. 
- ``presence_penalty`` affects the tendency for a model to repeat words found in the input. Negative values reward repetition, while positive values penalize repetition. The default value is zero; no particular preference if the response contains words from the input. 
- ``frequency_penalty`` affects the amount of repetition in the response. Negative values reward repetition, while positive values penalize repetition. The default value is zero. 

We will illustrate use of the ``max_tokens`` and ``temperature`` settings below with a new semantic function.

In [23]:
prompt = """
Write a funny punchline for the following joke:
```{{$input}}```
"""

input_text = "Why did the chicken cross the road?"

# Temperature of zero should always say "To get to the other side"
# Also, note smaller value for max_tokens (joke punchlines are shorter than emails)
joke_finisher_t0 = kernel.create_semantic_function(prompt, max_tokens=50, temperature=0)

print(input_text)
for i in range(5):
    output = joke_finisher_t0(input_text)
    print(f"Punchline {i+1}: {output}")

Why did the chicken cross the road?
Punchline 1: To get to the other side, duh!
Punchline 2: To get to the other side, duh!
Punchline 3: To get to the other side, duh!
Punchline 4: To get to the other side, duh!
Punchline 5: To get to the other side, duh!


In [24]:
# Temperature of 0.3: should start to see some variety in the joke punchlines
joke_finisher_t3 = kernel.create_semantic_function(prompt, max_tokens=50, temperature=0.3)

print(input_text)
for i in range(5):
    output = joke_finisher_t3(input_text)
    print(f"Punchline {i+1}: {output}")

Why did the chicken cross the road?
Punchline 1: To get to the other side...of the street, not life.
Punchline 2: To get to the silly punchline on the other side!
Punchline 3: To get to the other side, duh!
Punchline 4: To get to the other side, of course! (Classic, but still funny)
Punchline 5: To get to the other side, duh!


In [25]:
# Temperature of 0.9
joke_finisher_t9 = kernel.create_semantic_function(prompt, max_tokens=50, temperature=0.9)

print(input_text)
for i in range(5):
    output = joke_finisher_t9(input_text)
    print(f"Punchline {i+1}: {output}")

Why did the chicken cross the road?
Punchline 1: To prove to the possum that it could be done!
Punchline 2: To get to the other side, duh!
Punchline 3: To get to the idiot's house... Knock knock!
Punchline 4: To prove to the possum that it could be done!
Punchline 5: To prove it wasn't a chicken!


## Importing semantic functions from file

In SK, related semantic functions can be grouped together under one "Skill." The example semantic functions we have been modifying above are available under "ExampleSkill" in the skills directory of the workshop repository. The file structure is as follows: 

- Skills directory
  - Semantic skill directory ("ExampleSkill")
    - Semantic function directory ("EmailWriter")
      - skprompt.txt (text file with prompt string)
      - config.json (specify max_tokens, temperature, etc)
    - Another semantic function directory ("PunchlineWriter")
      - skprompt.txt
      - config.json
      
``skprompt.txt`` contains the prompt text and nothing else. ``config.json`` handles parameters such as max_tokens and temperature, and also contains a short description of the semantic function. 

This directory structure and these file names are required by SK, otherwise it will raise an error when you try to load in your semantic skills & functions. 

In [12]:
skills_directory = "./tmls_workshop_backend/skills"

In [13]:
# Loading in our ExampleSkill
example_skill = kernel.import_semantic_skill_from_directory(skills_directory, "ExampleSkill")

# Semantic functions contained in ExampleSkill are accessed using square brackets and the function name
output = example_skill['EmailWriter']('weekend road closures')
print(output)

Error: (<ErrorCodes.ServiceError: 6>, 'OpenAI service failed to complete the chat', RateLimitError(message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d00c775327ceffab41225c1fa1d209d1 in your message.)', http_status=429, request_id=None))


In [39]:
# Semantic functions contained in ExampleSkill are accessed as follows:
output = example_skill['PunchlineWriter']('Why is vanilla the best ice cream flavour?')
print(output)

Because it's the coolest!


## Native skills (python code)

SK allows for the creation of code-based or "native" skills and functions. There are several situations where a native function is a better choice than a semantic function:
- LLMs are not able to reliably work with numbers (imagine doing math by guessing likely digits!)
- Some tasks are easy to do with code and there is no need for an LLM
- Faster execution (no need to communicate with external model and wait for response)
- Not being billed by the token

The inputs and outputs can only be strings or the SKContext class. This will enable multiple skills/functions to be chained together to complete a multi-step request. The SKContext class is effectively a dictionary that can contain non-string arguments or multiple inputs.

In [53]:
from semantic_kernel.skill_definition import sk_function

# Our first native skill and function: adding a string of numbers together

class BasicMathSkill:

    @sk_function(
        description="Add a list of numbers together",
        name="addNumbers"
    )
    def add_numbers(self, input: str) -> str:
        try:
            return str(sum([float(x) for x in input.split()]))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

In [57]:
# Let's try adding 2, 3, and 4 together
math_skill = kernel.import_skill(BasicMathSkill())
output = math_skill["addNumbers"]("2 3 4")
print(output)

9.0


In [65]:
from semantic_kernel.skill_definition import sk_function_context_parameter
from semantic_kernel import SKContext

# "Advanced" math skill: dividing two numbers

class AdvancedMathSkill:

    @sk_function(
        description="Divide one number by another number",
        name="divideNumbers"
    )
    @sk_function_context_parameter(name="numerator", description="Number being divided")
    @sk_function_context_parameter(name="denominator", description="Number that numerator is divided by")
    def divide_numbers(self, context: SKContext) -> str:
        try:
            return str(context["numerator"] / context["denominator"]) 
        except ValueError as e:
            print(f"Invalid input {context['numerator']} {context['denominator']}")
            raise e
            
new_math_skill = kernel.import_skill(AdvancedMathSkill())

In [66]:
# 22/7 is a handy approximation for pi, so let's calculate that
context_variables = sk.ContextVariables(variables={
    "numerator": 22,
    "denominator": 7
})

output = new_math_skill["divideNumbers"].invoke(variables=context_variables).result
print(output)

3.142857142857143


## Planner

The planner is what bridges the gap between having a series of skills available to the kernel and automating the use of these skills to accomplish a given task. This is another power of LLMs: not only can we ask the model to write emails or tell jokes, but we can ask it to parse the input from a user and come up with a plan to generate a response. 

The planner prompt gives context to the model: the purpose of the application, the desire to parse the input into an action plan, and descriptions of all of the available skills. The skills loaded into the kernel are described automatically, so it is important to pick clear skill/function names and to fill out the description strings when creating them. 

The planner prompt asks the model for a formatted output plan (e.g. JSON or XML). Once the model has responded, code aspects of the planner class can handle the execution of the plain. The planner in the python version of SK is still under development, but we can have a quick look at it. 

In [67]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [79]:
ask = """
I want to come up with an election-related joke and send it in an email
"""
plan = await planner.create_plan_async(ask, kernel)

In [80]:
print(plan.generated_plan)

{
    "input": "election-related joke",
    "subtasks": [
        {"function": "ExampleSkill.PunchlineWriter"},
        {"function": "ExampleSkill.EmailWriter"}
    ]
}


In [81]:
results = await planner.execute_plan_async(plan, kernel)

In [82]:
print(results)

Subject: Clarification on Recent Personal Matters

Dear Neighbours,

I hope this email finds you well. I am writing to address recent rumors and speculation regarding my personal life. It has come to my attention that some of you may have heard that I recently broke up with my girlfriend. While I understand that personal matters can be of interest to some, I want to clarify that this decision was made solely because I wanted to focus on my running mate and the upcoming election.

As a public figure, I understand that my personal life may be subject to scrutiny. However, I want to assure you that my commitment to serving our community remains unwavering. I am fully dedicated to working with my running mate to address the issues that matter most to our constituents.

I appreciate your understanding and support during this time. If you have any questions or concerns, please do not hesitate to reach out to me directly.

Sincerely,

[Your Name]

P.S. If you are interested in learning more a

### Congratulations! You've learned the basics of Microsoft Semantic Kernel! :) 



